In [1]:
from PIL import Image
import numpy as np
from PIL import ImageFilter

feature minima von rot und grün
feature var r, g
feature mean von b

In [2]:
im = Image.open("positives/p01.png")
im.show()

In [18]:
np.mean(im.split()[0])
np.mean(im.split()[2])

150.64236111111111

In [27]:
extrema_pos =  []
for i in range(10,30):
    imm_pos = Image.open("positives/p" + str(i) + ".png")
    extrema_pos.append(imm_pos.getextrema())
    print(np.mean(imm_pos.split()[0]),np.mean(imm_pos.split()[1]),np.mean(imm_pos.split()[2]))
#    print(np.var(imm_pos.split()[0]),np.var(imm_pos.split()[1]),np.var(imm_pos.split()[2]))
extrema_pos

(190.22569444444446, 153.00347222222223, 155.40798611111111)
(159.68055555555554, 123.39930555555556, 130.77951388888889)
(191.859375, 152.47395833333334, 153.28645833333334)
(190.41493055555554, 151.90451388888889, 158.54166666666666)
(183.015625, 142.64236111111111, 148.81944444444446)
(183.96354166666666, 149.35069444444446, 151.484375)
(178.16319444444446, 145.73090277777777, 140.79340277777777)
(174.78993055555554, 141.36111111111111, 137.6875)
(170.36631944444446, 137.66145833333334, 135.22395833333334)
(160.87847222222223, 134.61458333333334, 128.25694444444446)
(177.12847222222223, 141.16145833333334, 145.04513888888889)
(184.13541666666666, 146.17534722222223, 142.94097222222223)
(175.40104166666666, 139.10416666666666, 139.24479166666666)
(171.47743055555554, 130.86805555555554, 140.19965277777777)
(165.04340277777777, 112.546875, 130.72048611111111)
(178.38368055555554, 126.97048611111111, 145.30208333333334)
(173.04861111111111, 126.109375, 133.47743055555554)
(168.91145833

[((116, 216), (45, 187), (115, 175)),
 ((93, 194), (50, 171), (91, 156)),
 ((113, 223), (43, 199), (114, 182)),
 ((91, 222), (61, 195), (113, 182)),
 ((121, 212), (56, 186), (100, 171)),
 ((110, 210), (62, 182), (127, 171)),
 ((81, 205), (59, 176), (113, 155)),
 ((89, 199), (36, 174), (101, 154)),
 ((74, 203), (39, 175), (100, 155)),
 ((59, 194), (38, 171), (97, 152)),
 ((77, 208), (62, 177), (109, 167)),
 ((101, 223), (37, 189), (95, 165)),
 ((112, 211), (55, 178), (103, 158)),
 ((87, 208), (48, 173), (117, 173)),
 ((61, 212), (31, 161), (78, 153)),
 ((69, 223), (43, 176), (113, 170)),
 ((124, 208), (37, 171), (86, 158)),
 ((79, 212), (28, 180), (91, 162)),
 ((93, 197), (48, 163), (81, 155)),
 ((68, 218), (38, 182), (117, 167))]

In [26]:
extrema_neg =  []
for i in range(10,30):
    imm_neg = Image.open("negatives/n" + str(i) + ".png")
    extrema_neg.append(imm_neg.getextrema())
 #   print(np.var(imm_neg.split()[0]),np.var(imm_neg.split()[1]),np.var(imm_neg.split()[2]))
    print(np.mean(imm_neg.split()[0]),np.mean(imm_neg.split()[1]),np.mean(imm_neg.split()[2]))
extrema_neg



(173.11284722222223, 125.29340277777777, 126.05208333333333)
(187.13368055555554, 154.58159722222223, 145.64583333333334)
(163.28645833333334, 124.62673611111111, 124.34548611111111)
(148.47916666666666, 110.76041666666667, 116.15798611111111)
(193.51388888888889, 163.47395833333334, 153.71180555555554)
(155.30381944444446, 106.51215277777777, 113.85416666666667)
(169.24305555555554, 137.13888888888889, 133.75520833333334)
(158.8125, 121.20486111111111, 122.29340277777777)
(166.54340277777777, 121.03993055555556, 123.31076388888889)
(155.05555555555554, 114.97222222222223, 119.27604166666667)
(163.33854166666666, 127.71875, 126.95659722222223)
(144.80381944444446, 105.44965277777777, 113.30555555555556)
(137.23263888888889, 100.67881944444444, 110.58506944444444)
(182.19618055555554, 158.80902777777777, 147.83854166666666)
(160.77430555555554, 124.11631944444444, 124.453125)
(157.36631944444446, 113.97743055555556, 117.30034722222223)
(158.00520833333334, 124.16666666666667, 133.232638

[((142, 198), (94, 160), (88, 156)),
 ((158, 207), (120, 173), (117, 167)),
 ((149, 181), (108, 143), (104, 143)),
 ((125, 165), (96, 123), (93, 133)),
 ((168, 207), (139, 174), (132, 171)),
 ((129, 178), (84, 129), (81, 139)),
 ((131, 195), (94, 168), (96, 165)),
 ((145, 177), (106, 146), (105, 145)),
 ((146, 194), (100, 153), (104, 154)),
 ((136, 174), (98, 134), (101, 137)),
 ((148, 177), (112, 142), (107, 145)),
 ((123, 188), (83, 152), (91, 149)),
 ((117, 190), (81, 161), (87, 154)),
 ((162, 197), (140, 174), (126, 162)),
 ((148, 174), (109, 138), (109, 136)),
 ((136, 179), (92, 137), (92, 145)),
 ((134, 199), (102, 163), (106, 156)),
 ((139, 171), (106, 137), (105, 137)),
 ((126, 171), (82, 119), (83, 122)),
 ((141, 203), (109, 177), (118, 174))]

In [53]:
im.filter(ImageFilter.FIND_EDGES).show()


In [4]:
im_neg = Image.open("negatives/n01.png")

In [58]:
np.mean(im_neg.histogram())

2.25

In [59]:
np.mean(im.histogram())

2.25

In [63]:
list(im_neg.getdata())


[(138, 106, 101),
 (138, 108, 100),
 (132, 102, 100),
 (136, 103, 113),
 (135, 101, 114),
 (130, 98, 108),
 (133, 103, 109),
 (135, 100, 97),
 (142, 101, 101),
 (146, 105, 108),
 (148, 109, 113),
 (149, 107, 112),
 (149, 108, 107),
 (153, 111, 111),
 (151, 111, 115),
 (152, 114, 127),
 (150, 112, 128),
 (152, 113, 121),
 (154, 116, 123),
 (146, 111, 113),
 (144, 108, 113),
 (142, 106, 112),
 (142, 105, 108),
 (147, 109, 113),
 (139, 104, 103),
 (141, 106, 100),
 (140, 104, 101),
 (140, 103, 107),
 (136, 101, 107),
 (133, 97, 99),
 (136, 100, 101),
 (139, 99, 102),
 (144, 102, 104),
 (146, 103, 105),
 (150, 106, 108),
 (152, 107, 109),
 (153, 107, 107),
 (156, 107, 112),
 (155, 111, 121),
 (151, 112, 126),
 (153, 111, 127),
 (151, 109, 119),
 (151, 113, 123),
 (146, 110, 122),
 (147, 110, 125),
 (147, 110, 121),
 (144, 110, 112),
 (144, 111, 114),
 (143, 107, 102),
 (142, 107, 102),
 (143, 106, 104),
 (143, 105, 98),
 (139, 101, 94),
 (142, 105, 105),
 (137, 103, 104),
 (137, 101, 106),